In [3]:
import os
from time import time
from feature_extraction import __get

path = r'/Users/loewi/Documents/Pre_Learn/classification/20news-bydate/'
os.chdir(path)
#print(os.getcwd())

t0 = time() 

newsgroups_train = __get('20news-bydate-train')
newsgroups_test = __get('20news-bydate-test')

duration = time() - t0
print('%0.2fs get data package ：）'%duration)

X_train, X_test = newsgroups_train['data'], newsgroups_test['data']
y_train, y_test = newsgroups_train['docs'], newsgroups_test['docs']
print('X & y prepared')

11.73s get data package ：）
X & y prepared


In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV,RandomizedSearchCV
from pprint import pprint
from sklearn import metrics
from sklearn.svm import SVC

In [12]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        n_features = 10000
                        )),
                
                ('clf', SVC( )),
                ])
parameters = { 
                'clf__kernel': ('linear', 'rbf'),
                }    
#SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#  max_iter=-1, probability=False, random_state=None, shrinking=True,
#  tol=0.001, verbose=False)

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = 5)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))   

Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__kernel': ('linear', 'rbf')}
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  4.4min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  6.4min finished


done in 465.245s

Best score: 0.619
Best parameters set:
	clf__kernel: 'linear'


In [13]:
import numpy as np
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
report(clf.cv_results_)       

Model with rank: 1
Mean validation score: 0.619 (std: 0.006)
Parameters: {'clf__kernel': 'linear'}

Model with rank: 2
Mean validation score: 0.062 (std: 0.018)
Parameters: {'clf__kernel': 'rbf'}



In [17]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        n_features = 10000
                        )),
                
                ('clf', SVC( 
                    kernel = 'rbf',
                )),
                ])
parameters = { 
                'clf__gamma': (1, 0.1, 0.01, 0.001, 0.00001, 10),
                'clf__C': (0.1, 1, 10, 100),
                }    

#SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#  max_iter=-1, probability=False, random_state=None, shrinking=True,
#  tol=0.001, verbose=False)

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = 5)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__C': (0.1, 1, 10, 100), 'clf__gamma': (1, 0.1, 0.01, 0.001, 1e-05, 10)}
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 22.7min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 66.4min finished


done in 4134.255s



In [20]:
import numpy as np
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
report(clf.cv_results_) 

Model with rank: 1
Mean validation score: 0.637 (std: 0.008)
Parameters: {'clf__C': 10, 'clf__gamma': 1}

Model with rank: 2
Mean validation score: 0.637 (std: 0.008)
Parameters: {'clf__C': 100, 'clf__gamma': 1}

Model with rank: 3
Mean validation score: 0.628 (std: 0.012)
Parameters: {'clf__C': 1, 'clf__gamma': 1}

Model with rank: 4
Mean validation score: 0.622 (std: 0.009)
Parameters: {'clf__C': 10, 'clf__gamma': 0.1}

Model with rank: 5
Mean validation score: 0.620 (std: 0.008)
Parameters: {'clf__C': 100, 'clf__gamma': 0.1}



In [21]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        n_features = 10000
                        )),
                
                ('clf', SVC( kernel = 'linear' )),
                ])
parameters = { 
                'clf__C': (0.025, 0.1, 1, 10, 100),    
                }    
#SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#  max_iter=-1, probability=False, random_state=None, shrinking=True,
#  tol=0.001, verbose=False)

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = 5)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__C': (0.025, 0.1, 1, 10, 100)}
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 12.0min finished


done in 799.559s



In [22]:
import numpy as np
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
report(clf.cv_results_) 

Model with rank: 1
Mean validation score: 0.619 (std: 0.006)
Parameters: {'clf__C': 1}

Model with rank: 2
Mean validation score: 0.581 (std: 0.014)
Parameters: {'clf__C': 10}

Model with rank: 3
Mean validation score: 0.570 (std: 0.015)
Parameters: {'clf__C': 100}

Model with rank: 4
Mean validation score: 0.467 (std: 0.004)
Parameters: {'clf__C': 0.1}

Model with rank: 5
Mean validation score: 0.120 (std: 0.014)
Parameters: {'clf__C': 0.025}



In [26]:
names = [ "Linear SVM", "RBF SVM"]

vectorizer = HashingVectorizer(stop_words='english')
X = vectorizer.fit_transform( X_train )
Xt= vectorizer.transform( X_test )
clf = [
    SVC(kernel="linear", C=1),
    SVC(gamma=10, C=1),
    ]
for name, clf in zip(names,clf):
    clf.fit(X, y_train)
    pred = clf.predict(Xt)
    score = metrics.accuracy_score(y_test, pred)
    print("%s accuracy:   %0.3f" % (name,score))

Linear SVM accuracy:   0.591
RBF SVM accuracy:   0.066
